In [1]:
import tensorflow as tf
import numpy as np

In [2]:
k = tf.constant([
    [1, 0, 1],
    [2, 1, 0],
    [0, 0, 1]
], dtype=tf.float32, name='k')
i = tf.constant([
    [4, 3, 1, 0],
    [2, 1, 0, 1],
    [1, 2, 4, 1],
    [3, 1, 0, 2]
], dtype=tf.float32, name='i')
kernel = tf.reshape(k, [3, 3, 1, 1], name='kernel')
image  = tf.reshape(i, [1, 4, 4, 1], name='image')

In [3]:
res0 = tf.nn.conv2d(image, kernel, [1, 1, 1, 1], "VALID")
res = tf.squeeze(res0)
# VALID means no padding
with tf.Session() as sess:
    print(sess.run(res0))
    print(sess.run(res))

[[[[14.]
   [ 6.]]

  [[ 6.]
   [12.]]]]
[[14.  6.]
 [ 6. 12.]]


In [6]:
with tf.Session() as sess:
    print(image.shape)
    print(kernel.shape)
    print(sess.run(res0))
    print(sess.run(res))

(1, 4, 4, 1)
(3, 3, 1, 1)
[[[[14.]
   [ 6.]]

  [[ 6.]
   [12.]]]]
[[14.  6.]
 [ 6. 12.]]


In [ ]:
#def extract_image_patches(images, ksizes, strides, rates, padding, name=None):     
    """Extract `patches` from `images` and put them in the "depth" output dimension.

    Args:                                                                                
    images: A `Tensor`. Must be one of the following types: `float32`, `float64`, `int32`, `uint8`, `int16`, `int8`, `int64`, `bfloat16`,            ……`uint16`, `half`, `uint32`, `uint64`.
      4-D Tensor with shape `[batch, in_rows, in_cols, depth]`.                        
    ksizes: A list of `ints` that has length `>= 4`.                                   
      The size of the sliding window for each dimension of `images`.                   
    strides: A list of `ints` that has length `>= 4`.                                  
      1-D of length 4. How far the centers of two consecutive patches are in           
      the images. Must be: `[1, stride_rows, stride_cols, 1]`.                         
    rates: A list of `ints` that has length `>= 4`.                                    
      1-D of length 4. Must be: `[1, rate_rows, rate_cols, 1]`. This is the            
      input stride, specifying how far two consecutive patch samples are in the    
      input. Equivalent to extracting patches with                                     
      `patch_sizes_eff = patch_sizes + (patch_sizes - 1) * (rates - 1)`, followed by
      subsampling them spatially by a factor of `rates`. This is equivalent to     
      `rate` in dilated (a.k.a. Atrous) convolutions.                                  
    padding: A `string` from: `"SAME", "VALID"`.                                       
      The type of padding algorithm to use.                                            

      We specify the size-related attributes as:                                       

      ```python                                                                        
            ksizes = [1, ksize_rows, ksize_cols, 1]                                    
            strides = [1, strides_rows, strides_cols, 1]                               
            rates = [1, rates_rows, rates_cols, 1]                                     
      ```                                                                              
    name: A name for the operation (optional).  

    Returns:                                                                         
    A `Tensor`. Has the same type as `images`.                                     
    """  

In [38]:
tf.extract_image_patches

In [ ]:
    image_patches = tf.extract_image_patches(images=images,
                                         ksizes=[1, filter_size, filter_size, 1],
                                         strides=[1, 1, 1, 1], 
                                             rates=[1, 1, 1, 1],
                                         padding='valid')

In [43]:
tf.shape(kernel)

<tf.Tensor 'Shape:0' shape=(4,) dtype=int32>

In [58]:
def get_flattned2(input, filter, strides, padding, use_cudnn_on_gpu=True, 
           data_format="NHWC", dilations=[1, 1, 1, 1], name=None):
    image_patches = tf.extract_image_patches(images=images,
                                         ksizes=[1, filter_size, filter_size, 1],
                                         strides=[1, 1, 1, 1], 
                                         rates=[1, 1, 1, 1],
                                         padding='valid')
    return image_patches
#test = get_flattned2(image,3)

with tf.Session() as sess:
    print(sess.run(tf.squeeze(image)))
    print(sess.run(tf.squeeze((kernel))))
    print(sess.run(test))

[3, 3, 1, 1]
[[4. 3. 1. 0.]
 [2. 1. 0. 1.]
 [1. 2. 4. 1.]
 [3. 1. 0. 2.]]
[[1. 0. 1.]
 [2. 1. 0.]
 [0. 0. 1.]]
[[[[0. 0. 0. 0. 4. 3. 0. 2. 1.]
   [0. 0. 0. 4. 3. 1. 2. 1. 0.]
   [0. 0. 0. 3. 1. 0. 1. 0. 1.]
   [0. 0. 0. 1. 0. 0. 0. 1. 0.]]

  [[0. 4. 3. 0. 2. 1. 0. 1. 2.]
   [4. 3. 1. 2. 1. 0. 1. 2. 4.]
   [3. 1. 0. 1. 0. 1. 2. 4. 1.]
   [1. 0. 0. 0. 1. 0. 4. 1. 0.]]

  [[0. 2. 1. 0. 1. 2. 0. 3. 1.]
   [2. 1. 0. 1. 2. 4. 3. 1. 0.]
   [1. 0. 1. 2. 4. 1. 1. 0. 2.]
   [0. 1. 0. 4. 1. 0. 0. 2. 0.]]

  [[0. 1. 2. 0. 3. 1. 0. 0. 0.]
   [1. 2. 4. 3. 1. 0. 0. 0. 0.]
   [2. 4. 1. 1. 0. 2. 0. 0. 0.]
   [4. 1. 0. 0. 2. 0. 0. 0. 0.]]]]


In [62]:
actual = tf.reduce_sum(tf.multiply(image, tf.reshape(kernel, [9])), 3, keep_dims=True)
expected = tf.nn.conv2d(image, kernel, strides=[1, 1, 1, 1], padding='SAME')


    

In [69]:
with tf.Session() as sess:
    print(sess.run(tf.squeeze(image)))
    print(sess.run(tf.squeeze(kernel)))
    print(sess.run(tf.squeeze(actual)))
    print(sess.run(tf.squeeze(expected)))

[[4. 3. 1. 0.]
 [2. 1. 0. 1.]
 [1. 2. 4. 1.]
 [3. 1. 0. 2.]]
[[1. 0. 1.]
 [2. 1. 0.]
 [0. 0. 1.]]
[[24. 18.  6.  0.]
 [12.  6.  0.  6.]
 [ 6. 12. 24.  6.]
 [18.  6.  0. 12.]]
[[ 5. 11.  8.  2.]
 [ 7. 14.  6.  2.]
 [ 3.  6. 12.  9.]
 [ 5. 12.  5.  6.]]


In [86]:
import tensorflow as tf
import numpy as np

image = np.arange(5 * 5 * 1).reshape(1, 5, 5, 1)

images = tf.convert_to_tensor(image.astype(np.float32))

filter_size = 3
sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], tf.float32)
sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])

image_patches = tf.extract_image_patches(images,
                                         [1, filter_size, filter_size, 1],
                                         [1, 1, 1, 1], [1, 1, 1, 1],
                                         padding='SAME')


actual = tf.reduce_sum(tf.multiply(image_patches, tf.reshape(sobel_x_filter, [9])), 3, keep_dims=True)
expected = tf.nn.conv2d(images, sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')

with tf.Session() as sess:
    print("input")
    print(sess.run(tf.squeeze(images)))
    
    print("kernel")
    print(sess.run(tf.squeeze(sobel_x_filter)))
    
    print("im2col-image")
    print("You see several matrices, each matrix has the elements to compute a "
          "full line_i of the output")
    print("each line_i of the one of these matrices "
          "produces one element j of a line in the output matrix")
    print(sess.run(tf.squeeze(image_patches)))
    
    print("im2col-filter")
    print(sess.run(tf.squeeze(tf.reshape(sobel_x_filter, [9]))))
    
    print("actual")
    print(sess.run(tf.squeeze(actual)))
    
    print("expected")
    print(sess.run(tf.squeeze(expected)))
    
    print("If result is zero, we are good:")
    print (sess.run(tf.reduce_sum(expected - actual)))

input
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]
kernel
[[-1.  0.  1.]
 [-2.  0.  2.]
 [-1.  0.  1.]]
im2col-image
You see several matrices, each matrix has the elements to compute a full line_i of the output
each line_i of the one of these matrices produces one element j of a line in the output matrix
[[[ 0.  0.  0.  0.  0.  1.  0.  5.  6.]
  [ 0.  0.  0.  0.  1.  2.  5.  6.  7.]
  [ 0.  0.  0.  1.  2.  3.  6.  7.  8.]
  [ 0.  0.  0.  2.  3.  4.  7.  8.  9.]
  [ 0.  0.  0.  3.  4.  0.  8.  9.  0.]]

 [[ 0.  0.  1.  0.  5.  6.  0. 10. 11.]
  [ 0.  1.  2.  5.  6.  7. 10. 11. 12.]
  [ 1.  2.  3.  6.  7.  8. 11. 12. 13.]
  [ 2.  3.  4.  7.  8.  9. 12. 13. 14.]
  [ 3.  4.  0.  8.  9.  0. 13. 14.  0.]]

 [[ 0.  5.  6.  0. 10. 11.  0. 15. 16.]
  [ 5.  6.  7. 10. 11. 12. 15. 16. 17.]
  [ 6.  7.  8. 11. 12. 13. 16. 17. 18.]
  [ 7.  8.  9. 12. 13. 14. 17. 18. 19.]
  [ 8.  9.  0. 13. 14.  0. 18. 19.  0.]]

 [[ 0. 10. 11.  0

In [7]:
# Experiment with reshaping the tensor:
data_3d = np.arange(27).reshape((3,3,3))
data_2d = data_3d.swapaxes(1,2).reshape(3,-1)

In [ ]:
zero = tf.constant(0, dtype=tf.float32)
where = tf.not_equal(A, zero)

In [116]:
def get_dim_sparsity(input_tensor,dim=0):
    x_dim_size = x.shape[dim]
    dim_nz=tf.count_nonzero(x, dim)
    dim_density = dim_nz/x_dim_size
    sparsity_tensor = 1-dim_density
    return sparsity_tensor

def get_row_sparsity(input_tensor):
    x_row_size = x.shape[0]
    column_nz=tf.count_nonzero(x, 0)
    column_density = column_nz/x_row_size
    sparsity_tensor = 1-column_density
    return sparsity_tensor

def get_col_sparsity(input_tensor):
    x_col_size = x.shape[1]
    column_nz=tf.count_nonzero(x, 1)
    column_density = column_nz/x_col_size
    sparsity_tensor = 1-column_density
    return sparsity_tensor


x = tf.constant([[[0, 1, 0], 
                  [1, 1, 0]],
                 [[0, 1, 0], 
                  [1, 1, 0]]])
column_sparsity1 = get_col_sparsity(x)

dim_sparsity2 = get_dim_sparsity(x)
with tf.Session() as sess:
    print(sess.run(column_sparsity1))
    print()
    print(sess.run(dim_sparsity2))


[[0.5 0.  1. ]
 [0.5 0.  1. ]]

[[1. 0. 1.]
 [0. 0. 1.]]


In [41]:
def reshape_image_tensor(image_tensor):
    """Reshapes tensor batch_size x H x W x Channels."""
    
    reshaped = image_tensor
    
    
    return reshaped

x = tf.constant([[[1, 2, 3], 
                  [4, 5, 6]],
                 [[7, 8, 9], 
                  [0, 0, 0]]])

last_indice = x.get_shape().as_list()[0] - 1
res = tf.slice(x, [0, 0, 0], [1, -1, -1])

with tf.Session() as sess:
    print(sess.run(res))
    print()
    print(x.shape)

[[[1 2 3]
  [4 5 6]]]

(2, 2, 3)


In [120]:
# Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
# And a and a filter / kernel tensor of shape `[filter_height, filter_width, in_channels, out_channels]`, this op
x1 = tf.constant(
[ # 2,4,4,1
    [
        [4, 3, 1, 0],
        [2, 1, 0, 1],
        [1, 2, 4, 1],
        [3, 1, 0, 2]
    ],
    [
        [1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3],
        [4, 5, 6, 7]
    ],
],dtype=tf.float32, name='i')

x = tf.constant(
[ # 2, 4, 4, 2
    [
        [[4, 3], [1, 0],[1, 1], [2, 2]],
        [[2, 1], [0, 1],[2, 1], [0, 1]],  
        [[1, 2], [4, 1],[2, 1], [0, 1]],
        [[3, 1], [0, 2],[2, 1], [0, 1]]
    ],
    [
        [[1, 1], [1, 1],[2, 1], [0, 1]],
        [[2, 2], [2, 2],[2, 1], [0, 1]],
        [[3, 3], [3, 3],[2, 1], [0, 1]],
        [[4, 5], [6, 7],[2, 1], [0, 1]]
    ],
],dtype=tf.float32, name='i')

#x  = tf.reshape(x, [2, 4, 4, 1], name='image')
x  = tf.reshape(x, [2, 4, 4, 2], name='image')

def get_image_patches(input_tensor, filter_size, filter_channels=1,n_of_filters=1):
    image_patches = tf.extract_image_patches(input_tensor,
                                         [1,filter_size, filter_size, 1],
                                         [1, 1, 1, 1], [1, 1, 1, 1],
                                         padding='SAME')
    return image_patches

image_patches = get_image_patches(x,2)
with tf.Session() as sess:
    print(sess.run(tf.squeeze(x)))
    print()
    
    print(sess.run(image_patches))

[[[[4. 3.]
   [1. 0.]
   [1. 1.]
   [2. 2.]]

  [[2. 1.]
   [0. 1.]
   [2. 1.]
   [0. 1.]]

  [[1. 2.]
   [4. 1.]
   [2. 1.]
   [0. 1.]]

  [[3. 1.]
   [0. 2.]
   [2. 1.]
   [0. 1.]]]


 [[[1. 1.]
   [1. 1.]
   [2. 1.]
   [0. 1.]]

  [[2. 2.]
   [2. 2.]
   [2. 1.]
   [0. 1.]]

  [[3. 3.]
   [3. 3.]
   [2. 1.]
   [0. 1.]]

  [[4. 5.]
   [6. 7.]
   [2. 1.]
   [0. 1.]]]]

[[[[4. 3. 1. 0. 2. 1. 0. 1.]
   [1. 0. 1. 1. 0. 1. 2. 1.]
   [1. 1. 2. 2. 2. 1. 0. 1.]
   [2. 2. 0. 0. 0. 1. 0. 0.]]

  [[2. 1. 0. 1. 1. 2. 4. 1.]
   [0. 1. 2. 1. 4. 1. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[1. 2. 4. 1. 3. 1. 0. 2.]
   [4. 1. 2. 1. 0. 2. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[3. 1. 0. 2. 0. 0. 0. 0.]
   [0. 2. 2. 1. 0. 0. 0. 0.]
   [2. 1. 0. 1. 0. 0. 0. 0.]
   [0. 1. 0. 0. 0. 0. 0. 0.]]]


 [[[1. 1. 1. 1. 2. 2. 2. 2.]
   [1. 1. 2. 1. 2. 2. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[2. 2. 2. 2. 3. 3. 3. 3.]
   [2. 

In [151]:
def get_dim_sparsity(input_tensor,dim=0):
    dim_size = input_tensor.shape[dim]
    dim_nz=tf.count_nonzero(input_tensor, dim)
    dim_density = dim_nz/dim_size
    sparsity_tensor = 1-dim_density
    return sparsity_tensor
dim=3


image_patches = get_image_patches(x,2)

sparsity_out = get_dim_sparsity(image_patches,dim)

with tf.Session() as sess: 
    print(sess.run(image_patches))
    print(sess.run(sparsity_out))

[[[[4. 3. 1. 0. 2. 1. 0. 1.]
   [1. 0. 1. 1. 0. 1. 2. 1.]
   [1. 1. 2. 2. 2. 1. 0. 1.]
   [2. 2. 0. 0. 0. 1. 0. 0.]]

  [[2. 1. 0. 1. 1. 2. 4. 1.]
   [0. 1. 2. 1. 4. 1. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[1. 2. 4. 1. 3. 1. 0. 2.]
   [4. 1. 2. 1. 0. 2. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[3. 1. 0. 2. 0. 0. 0. 0.]
   [0. 2. 2. 1. 0. 0. 0. 0.]
   [2. 1. 0. 1. 0. 0. 0. 0.]
   [0. 1. 0. 0. 0. 0. 0. 0.]]]


 [[[1. 1. 1. 1. 2. 2. 2. 2.]
   [1. 1. 2. 1. 2. 2. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[2. 2. 2. 2. 3. 3. 3. 3.]
   [2. 2. 2. 1. 3. 3. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[3. 3. 3. 3. 4. 5. 6. 7.]
   [3. 3. 2. 1. 6. 7. 2. 1.]
   [2. 1. 0. 1. 2. 1. 0. 1.]
   [0. 1. 0. 0. 0. 1. 0. 0.]]

  [[4. 5. 6. 7. 0. 0. 0. 0.]
   [6. 7. 2. 1. 0. 0. 0. 0.]
   [2. 1. 0. 1. 0. 0. 0. 0.]
   [0. 1. 0. 0. 0. 0. 0. 0.]]]]
[[[0.25  0.25  0.125 0.625]
  [0.125 0.125 0.25  0.75